# Sistemas planetarios detectados por velocidad radial


### Antes de nada abrir jupyter desde la terminal con:
jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10


In [1]:
import pandas as pd
import numpy as np
from numpy import arange,cos,sin,pi,sqrt,arcsin

%matplotlib inline
import matplotlib.pyplot as plt
from astropy import constants as C


In [2]:
#carga de datos planetarios obtenidos en la base de datos de Gaia
exoplanet=pd.read_csv("Exoplanetas detectables.csv")
exoplanet
# COLUMN pl_name:        Planet Name
# COLUMN hostname:       Host Name
# COLUMN discoverymethod: Discovery Method
# COLUMN pl_orbper:      Orbital Period [days]
# COLUMN pl_orbsmax:     Orbit Semi-Major Axis [au])
# COLUMN pl_rade:        Planet Radius [Earth Radius]
# COLUMN pl_bmasse:      Planet Mass or Mass*sin(i) [Earth Mass]
# COLUMN pl_orbeccen:    Eccentricity
# COLUMN st_rad:         Stellar Radius [Solar Radius]
# COLUMN st_mass:        Stellar Mass [Solar mass]
# COLUMN rastr:          RA [sexagesimal]
# COLUMN ra:             RA [deg]
# COLUMN decstr:         Dec [sexagesimal]
# COLUMN dec:            Dec [deg]
# COLUMN sy_dist:        Distance [pc]
# COLUMN sy_plx:         Parallax [mas]
#AÑADIR RADIAL VELOCITY AMPLITUDE [M/S] !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

,Unnamed: 0,pl_name,hostname,discoverymethod,pl_controv_flag,pl_orbper,pl_orbsmax,pl_radj,pl_bmassj,pl_orbeccen,...,st_rad,st_mass,rastr,ra,decstr,dec,sy_dist,sy_plx,sy_gaiamag,Rhomax
0,0,11 Com b,11 Com,Radial Velocity,0,326.030000,1.290000,1.08,19.4000,0.2310,...,19.00,2.70,12h20m42.91s,185.178779,+17d47m35.71s,17.793252,93.1846,10.71040,4.44038,352.644737
1,1,11 UMi b,11 UMi,Radial Velocity,0,516.219970,1.530000,1.09,14.7400,0.0800,...,29.79,2.78,15h17m05.90s,229.274595,+71d49m26.19s,71.823943,125.3210,7.95388,4.56216,349.345514
2,2,14 And b,14 And,Radial Velocity,0,185.840000,0.830000,1.15,4.8000,0.0000,...,11.00,2.20,23h31m17.80s,352.824150,+39d14m09.01s,39.235837,75.4392,13.22890,4.91781,343.267712
3,3,14 Her b,14 Her,Radial Velocity,0,1773.400020,2.930000,1.15,4.6600,0.3700,...,0.93,0.90,16h10m24.50s,242.602101,+43d48m58.90s,43.816362,17.9323,55.73630,6.38300,11789.421340
4,4,16 Cyg B b,16 Cyg B,Radial Velocity,0,798.500000,1.660000,1.20,1.7800,0.6800,...,1.13,1.08,19h41m51.75s,295.465642,+50d31m00.57s,50.516824,21.1397,47.27540,6.06428,5202.606455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902,912,ups And b,ups And,Radial Velocity,0,4.617033,0.059222,1.25,0.6876,0.0215,...,1.56,1.30,01h36m47.60s,24.198353,+41d24m13.73s,41.403815,13.4054,74.57110,3.98687,233.571039
903,913,ups And c,ups And,Radial Velocity,0,241.258000,0.827774,1.10,13.9800,0.2596,...,1.56,1.30,01h36m47.60s,24.198353,+41d24m13.73s,41.403815,13.4054,74.57110,3.98687,3264.611368
904,914,ups And d,ups And,Radial Velocity,0,1276.460000,2.513290,1.11,10.2500,0.2987,...,1.56,1.30,01h36m47.60s,24.198353,+41d24m13.73s,41.403815,13.4054,74.57110,3.98687,9912.468337
905,915,ups Leo b,ups Leo,Radial Velocity,0,385.200000,1.180000,1.27,0.5100,0.3200,...,11.22,1.48,11h36m56.93s,174.237219,-00d49m24.83s,-0.823564,52.5973,18.99390,4.03040,1042.490109


In [3]:
#describimos las constantes que se van a ncesitar
Pseg=exoplanet.pl_orbper*24*3600 #periodo orbital en dias
mtot=exoplanet.st_mass*C.M_sun.value #masa de la estrella en masas solares
dtot=exoplanet.sy_dist*C.pc.value #distancia al sol en metros
m2sini=exoplanet.pl_bmassj*C.M_jup.value #masa mínima del planeta en masas de jupiter
m2sini=np.array(m2sini)
# *m2sini/mtot1/sin(i))
e=exoplanet.pl_orbeccen #excenricidad de la órbita

#Constantes
G1=C.G.value

# e=0 #suponiendo orbita circular
e=e.fillna(0)
omega2=pi/2
u=len(exoplanet.index)  #numero de planetas detectados por velocidad radial
U=arange(u)

In [4]:
#Cálculo de E. Si no se supone orbita circular E tiene un valor que puede ser calculado 
#metodo iterativo con la ecuación de kepler E-sin(E)=M con M la anomalía media
# Se porocede a coger el valor de ka anomalia media como 0.25*2*pi suponiendo la cuadratura.
#Esto se debe a que en este valor sera el maximo que obtengamos de la separacion
#vamos a probar con el metodo de las aproxi aciones sucesivas
AnomExce=[]
for exe in U:
    M=0.25*2*pi
    E_0=M
    E_1=M+e[exe]*sin(E_0)
    while abs(E_1-E_0)>0.0001:
        E_0=E_1
        E_1=M+e[exe]*sin(E_0)
    AnomExce.append(E_1)
    

In [5]:
#lo primero de todo es definir un limite inferior para i para cada planeta que sera si las masa del plaenta supera las 72 masas
#de jupiter, es decir, sería una estrella
i_min=[]
for l in U:
    i_min.append(arcsin(m2sini[l]/(72*C.M_jup.value)))

In [7]:
#arrays

E=AnomExce  #anomalia excentrica

Omega2=np.linspace(0,2*pi,100)
i=np.linspace(np.amin(i_min),2*pi,100)


In [8]:
new_i,new_Omega2=np.meshgrid(i,Omega2)

In [10]:
#Coordenadas rectangulares cojn excentricidad diferente de 0
X=np.cos(E)-e
Y=sqrt(1-e**2)*np.sin(E)

In [11]:
#ponemos todas las funciones para el calculo de los elementos de Thiele-Innes
def Af(atot,Omega2,omega2,i):
    return atot*(cos(Omega2)*cos(omega2)-sin(Omega2)*sin(omega2)*cos(i))
def Bf(atot,Omega2,omega2,i):
    return atot*(sin(Omega2)*cos(omega2)+cos(Omega2)*sin(omega2)*cos(i))
def Ff(atot,Omega2,omega2,i):
    return atot*(-cos(Omega2)*sin(omega2)-sin(Omega2)*cos(omega2)*cos(i))
def Gf(atot,Omega2,omega2,i):
    return atot*(-sin(Omega2)*sin(omega2)+cos(Omega2)*cos(omega2)*cos(i))



In [12]:
#VECTORIZACION DE FUNCIONES
A=np.vectorize(Af)
B=np.vectorize(Bf)
G=np.vectorize(Ff)
F=np.vectorize(Gf)


In [13]:
#Tercera ley de kepler para calcular el semieje mayor en metros
def atota(P,mtot):
    return (P**2*G1*mtot/(4*pi**2))**(1/3)

In [14]:
atotal=np.vectorize(atota)

In [15]:
atot=(atotal(Pseg,mtot)/dtot*180/pi*3600e6)*m2sini/mtot  #sale en muas

In [17]:

#Calculo de los elementos de Thiele-Innes. en nuestra base de datos tenemos 915 planetas el resultado debera ser una lista 
# con 915 arrays, uno por planeta.
a1=[]
b1=[]
f1=[]
g1=[]
for m in arange(len(atot)):
    primea=Af(atot[m],new_Omega2,omega2,new_i)
    primeb=Bf(atot[m],new_Omega2,omega2,new_i)
    primef=Ff(atot[m],new_Omega2,omega2,new_i)
    primeg=Gf(atot[m],new_Omega2,omega2,new_i)
    a1.append(primea)
    b1.append(primeb)
    f1.append(primef)
    g1.append(primeg)





## Delta-alfa y Delta-delta


In [18]:
#obtención de delta-delta y delta-alpha y su vectorizacion
def deldel(A,X,F,Y):
    return A*X+F*Y
def delalp(B,X,G,Y):
    return B*X+G*Y        
alpa=np.vectorize(delalp)
delpa=np.vectorize(deldel)

### Delta-Alfa

In [26]:
alfa1,delta1=[],[]
for m  in U:
    alfa1.append(delalp(a1[m],X[m],f1[m],Y[m]))
    delta1.append(deldel(b1[m],X[m],g1[m],Y[m]))
alfa1 = np.array(alfa1)
delta1 = np.array(delta1)


In [27]:
#AHORA PROCEDEMOS A DEFINIR RHO 
def rho(alfa,delta):
    return sqrt(alfa**2+delta**2)
vec_rho=np.vectorize(rho)

In [28]:
astro_sig=[]
for vals in U:
    astro_sig.append(rho(alfa1[vals],delta1[vals]))


In [29]:
# vamos a eliminar el valor de astro_sig[x][0] porque el limite cogido de i no es el adecuado por lo que aumenta en exceso la medida y el error
mod_astro=[]
for vals_err in U:
    for i_vals in arange(len(i)):
        mod_astro.append(np.delete(astro_sig[vals_err][i_vals],[0]))



In [30]:
max_sig=[]
for p in (U*100):
    max_sig.append(np.amax(mod_astro[p]))


In [33]:
#ahora lo que queda es ver si Gaia resolvera estos planetas o no.
#Para ello compararemos la maxima separacion y dividiremos en dos bases de datos
gaialim=20 #muas
Max_Sig=pd.DataFrame(max_sig)
Max_Sig.columns=['Max_Sig']
Max_Sig.index=exoplanet.pl_name
not_solved=Max_Sig[Max_Sig['Max_Sig']<gaialim].index
Max_Sig.drop(not_solved, inplace=True)
# for sig in U:
#     while abs(max_sig[sig]>20):
Max_Sig.to_csv('Exoplanetas detectables por GAIA.csv')      
Max_Sig

,Max_Sig
pl_name,
11 Com b,99.918763
11 UMi b,71.974572
14 And b,22.884514
14 Her b,859.274514
16 Cyg B b,175.950178
...,...
omi UMa b,92.583049
psi 1 Dra B b,273.876018
tau Gem b,89.110911
